In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


In [ ]:
chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 세계적인 수준의 요리사 입니다. 구하기 쉬운 재료로 모든 종류의 요리에 대해 따라하기 쉬운 레시피를 만듭니다.."),
        ("human", "나는 {cuisine} 음식을 요리하고 시어요."),
        
    ]
)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 전통적인 요리법을 채식주의 식단으로 만드는 채식전문 요리사 입니다. 대체 재료를 찾고 그 준비 과정을 설명합니다. 레시피를 획기적으로 수정하지 마세요. 음식에 대한 대안이 없다면 대체하는 방법을 모른다고 말하세요."),
        ("human", "{recipe}")    
    ]
)


"""
chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cusine with easy to find ingredients"),
        ("human", "I want to cook {cuisine} food."),
        
    ]
)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don,t redically modify the recipe. If there is no alternative for a food just say you don,t know how to replace it."),
        ("human", "{recipe}")    
    ]
)
"""


In [ ]:
chef_chain = chef_prompt | chat 

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke(
    {
        "cuisine": "중국요리"
    }
)